# Item 70: Use `Queue` to Coordinate Work Between Threads


In [ ]:
from collections import deque
from threading import Lock, Thread
import time

class MyQueue:
    def __init__(self):
        self.items = deque()
        self.lock = Lock()
        
    def put(self, item):
        with self.lock:
            self.items.append(item)
            
    def get(self):
        with self.lock:
            return self.items.popleft()
        
class Worker(Thread):
    def __init__(self, func, in_queue, out_queue):
        super().__init__()
        self.func = func 
        self.in_queue = in_queue
        self.out_queue = out_queue
        self.polled_count = 0
        self.work_done = 0
        
    def run(self):
        while True:
            self.polled_count += 1
            try:
                item = self.in_queue.get()
            except IndexError:
                time.sleep(0.01) # No work to do 
            else:
                result = self.func(item)
                self.out_queue.put(result)
                self.work_done += 1

def download(item):
    return item

def resize(item):
    return item

def upload(item):
    return item
    
download_queue = MyQueue()
resize_queue = MyQueue()
upload_queue = MyQueue()
done_queue = MyQueue()
threads = [
    Worker(download, download_queue, resize_queue),
    Worker(resize, resize_queue, upload_queue),
    Worker(upload, upload_queue, done_queue),
]

for thread in threads:
    thread.start()

for _ in range(1000):
    download_queue.put(object())
    
while len(done_queue.items) < 1000:
    # Do something useful while waiting
    ...
    
processed = len(done_queue.items)
polled = sum(t.polled_count for t in threads)
print(f"Processed {processed} items after "
      f"polling {polled} times")
    
    

Processed 1000 items after polling 3011 times


以上实现方式，有三个问题：
1. First, determining that all of the input work is complete requires yet another busy wait on the `done_queue`. 
2. Second, in `Worker`, the `run` method will execute forever in its busy loop.(There's no obvious way to signal to a worker thread that it's time to exit.)
3. Third, and worst of all, a backup in the pipeline can cause the program to crash arbitrarily.
